In [12]:
from google.cloud import bigquery
from google.oauth2 import service_account
import json
import os
import pandas as pd

# TensorFlow e tf.keras
import tensorflow as tf
from tensorflow import keras
from sklearn.linear_model import LogisticRegression

# Librariesauxiliares
import pandas as pd
import numpy as np

In [2]:
# Configurando conexão com BigQuery
def bigQueryConnect(key): 
    cwd = os.getcwd()
    keyPath = cwd+'/'+key
    with open(keyPath) as f:
        key = json.load(f)
    credentials = service_account.Credentials.from_service_account_file(keyPath)
    client = bigquery.Client(credentials= credentials,project=key['project_id'])
    return client

In [14]:
client = bigQueryConnect('keys/service-account-key.json')
job = client.query( """
    SELECT
    EXTRACT(DAYOFWEEK FROM data_busca) as dia_semana,
    EXTRACT(MONTH FROM data_busca) as mes,
    (CASE companhia_aerea WHEN 'GOL' THEN 1 ELSE 0 END) AS companhia_gol,
    (CASE companhia_aerea WHEN 'TAM' THEN 1 ELSE 0 END) AS companhia_tam,
    (CASE companhia_aerea WHEN 'AZUL' THEN 1 ELSE 0 END) AS companhia_azul,
    (CASE companhia_aerea WHEN 'AVIANCA' THEN 1 ELSE 0 END) AS companhia_avianca,
    qtd_voos,
    qtd_voos_recebidos,
    (CASE tipo_de_voo WHEN 'Ida' THEN 1 ELSE 0 END) AS ida_ou_volta,
    dias_entre_a_viagem,
    (CASE WHEN qtd_melhores_precos_na_mm > qtd_melhores_precos_nas_cias THEN 1 ELSE 0 END) AS mais_precos_mais_barato_mm

    FROM `maxmilhas-teste.teste.buscas`
    WHERE (qtd_melhores_precos_na_mm > 0 OR  qtd_melhores_precos_nas_cias > 0)
    
    """)
resultado = job.result()
dados = resultado.to_dataframe()
dados.head()

,dia_semana,mes,companhia_gol,companhia_tam,companhia_azul,companhia_avianca,qtd_voos,qtd_voos_recebidos,ida_ou_volta,dias_entre_a_viagem,mais_precos_mais_barato_mm
0,4,1,0,1,0,0,51,50,0,6,1
1,2,1,0,1,0,0,47,46,1,0,1
2,2,1,0,0,1,0,8,13,0,4,0
3,2,1,1,0,0,0,15,15,1,0,1
4,2,1,0,0,1,0,6,6,1,0,0


In [15]:
input = dados[1:int(len(dados)*0.7)].reset_index(drop=True)
predict = dados[int(len(dados)*0.7):len(dados)].reset_index(drop=True)
features = [
    'dia_semana',
    'mes','companhia_gol',
    'companhia_azul',
    'companhia_tam',
    'companhia_avianca',
    'qtd_voos',
    'qtd_voos_recebidos',
    'ida_ou_volta',
    'dias_entre_a_viagem'
]
prediction_class = 'mais_precos_mais_barato_mm'

In [17]:
inputX = input[features].to_numpy()
inputY = input[prediction_class].to_numpy()

predictX = predict[features].to_numpy()
predictY = predict[prediction_class].to_numpy()

In [18]:
model = keras.Sequential([
    keras.layers.Dense(10, activation='linear'),
    keras.layers.Dense(10, activation='linear'),
    keras.layers.Dense(10, activation='linear'),
    keras.layers.Dense(10, activation='linear'),
    keras.layers.Dense(10, activation='linear'),
    keras.layers.Dense(10, activation='linear'),
    keras.layers.Dense(10, activation='linear'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [19]:
model.compile(optimizer='adam',
      loss='binary_crossentropy',
      metrics=['accuracy'])

In [21]:
model.fit(inputX, inputY, epochs=1000)

- accuracy: 0.8814
Epoch 816/1000
506/506 [==============================] - 0s 128us/sample - loss: 0.3059 - accuracy: 0.8794
Epoch 817/1000
506/506 [==============================] - 0s 141us/sample - loss: 0.3064 - accuracy: 0.8834
Epoch 818/1000
506/506 [==============================] - 0s 125us/sample - loss: 0.3057 - accuracy: 0.8755
Epoch 819/1000
506/506 [==============================] - 0s 137us/sample - loss: 0.3057 - accuracy: 0.8794
Epoch 820/1000
506/506 [==============================] - 0s 164us/sample - loss: 0.3146 - accuracy: 0.8834
Epoch 821/1000
506/506 [==============================] - 0s 162us/sample - loss: 0.3104 - accuracy: 0.8715
Epoch 822/1000
506/506 [==============================] - 0s 149us/sample - loss: 0.3071 - accuracy: 0.8794
Epoch 823/1000
506/506 [==============================] - 0s 164us/sample - loss: 0.3124 - accuracy: 0.8814
Epoch 824/1000
506/506 [==============================] - 0s 150us/sample - loss: 0.3062 - accuracy: 0.8755
Epoch 825

In [22]:
test_loss, test_acc = model.evaluate(predictX,  predictY, verbose=2)

218/1 - 0s - loss: 0.3563 - accuracy: 0.8761


In [23]:
y_pred_NN = model.predict(predictX, verbose=1).flatten().round()
1-sum(abs(y_pred_NN-predictY))/len(predictY)

218/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

0.8761467889908257

In [24]:
# instantiate the model (using the default parameters)
logreg = LogisticRegression()

# fit the model with data
logreg.fit(inputX,inputY)

#
y_pred=logreg.predict(predictX)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [25]:
1-sum(abs(y_pred-predictY))/len(predictY)

0.8669724770642202